# Homework 4

Team mate with Linyao Li

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets

In [2]:
import pandas as pd

ufos = pd.read_csv("../dataset/ufo-scrubbed-geocoded-time-standardized-00.csv",
                  names = ["date", "city", "state", "country",
                          "shape", "duration_seconds", "duration",
                          "comment", "report_date", "latitude", "longitude"],
                  parse_dates = ["date", "report_date"])

# you might get a memory warning thing, its just not deprecated correctly
#  try not to panic :D

In [25]:
ufos['Year'] = ufos['date'].apply(lambda x: "%d" % (x.year))
ufos['Report_Year'] = ufos['report_date'].apply(lambda x: "%d" % (x.year))

In [3]:
ufos = ufos.fillna({'country':'unknown', 'state': 'unknown', 'shape':'unknown'})

In [21]:
nsamples = 2000
downSampleMask = np.random.choice(range(len(ufos)-1), nsamples, replace=False)
downSampleMask # take a look

array([47562, 67765, 19937, ...,  4219, 25288,  4751])

In [26]:
ufosDS = ufos.loc[downSampleMask]

In [7]:
import matplotlib.colors as mpl_colors
from IPython.display import display, clear_output
%config InlineBackend.close_figures=False 

In [9]:
ufosDS.shape

(5000, 11)

In [ ]:
col_names

In [41]:
plt.ioff()
plt.close('all')  # if you get a "too many figures open" warning
# fig = plt.figure(figsize=(8,8))
# add axes by hand like last week
# order here is: left, bottom, width, height

fig, ax = plt.subplots(figsize=(8, 8))

ufosDS.hist(column='latitude', ax=ax, bins=10)

out = ipywidgets.Output()  # NEW WIDGET CALL

irange = ipywidgets.IntSlider(min=5, max=25, step=1, value=10)

col_names = [
    'Year',
    'state',
    'country',
    'shape',
    'duration_seconds',
    'Report_Year',
    'latitude',
    'longitude']
dropdown = ipywidgets.Dropdown(
    options=col_names, index=col_names.index('latitude'))

controls = ipywidgets.VBox([dropdown, irange])
controls.layout.top = '200px'  # UPDATED
hbox = ipywidgets.HBox([out, controls])

display(hbox)


def show_plt(col):  # Reuse Plot Function
    for a in fig.axes:
        fig.delaxes(a)
        ax = fig.add_axes([0.0, 0.15, 1.0, 0.8])
    if ufos[col].dtype != 'O':  # numberical
        ufosDS.hist(column=col, ax=ax, bins=irange.value)
    else:  # catagorical
        ufosDS[col].value_counts().plot(kind='bar', ax=ax)
        controls = ipywidgets.VBox([dropdown])
    if col in ['shape', 'country']:
        plt.xticks(rotation=30)


def updateBin(change):
    if change is not None:
        col = col_names[dropdown.index]
        show_plt(col)
    with out:
        clear_output(wait=True)
        display(fig)


def updateContent(change):
    if change is not None and change['name'] == 'value':
        col = col_names[change['owner'].index]
        show_plt(col)
    with out:
        clear_output(wait=True)
        display(fig)


irange.observe(updateBin)
dropdown.observe(updateContent, names='value')
updateBin(None)
updateContent(None)

Because it is a histogram, so pick a certain column is enough, the hist function will help users aggregate value based on the number of values. For numerical data, the hist function is enough, but for categorical column, we should do some aggregation manually. With the value_counts function, we can calculate the count easily. Here are explanations for the column values I chose and the inner logic in visualization.

1. I delete the city and comment fields because there are too much diverse values which makes it hard to show in a histogram. The duration data field is strange in description so I also delete it.

2. I Edited two date fields in year range and aggregate based on each year to reduce the cluster number.

3. Only allow edit bin number for numerical data because merging any categorical data would cause confusion.

4. Reuse plot function to reduce repeat codes.

In [40]:
%config InlineBackend.close_figures=False
plt.ioff()
plt.close('all') # if you get a "too many figures open" warning
fig = plt.figure(figsize=(12,10))
# add axes by hand like last week
# order here is: left, bottom, width, height
ax = fig.add_axes([0.0, 0.15, 1.0, 0.8])
image = ax.scatter(ufosDS['longitude'],ufosDS['latitude'],c=np.log10(ufosDS['duration_seconds']))

dropdown_x = ipywidgets.Dropdown(options=list(ufosDS.columns),index=list(ufosDS.columns).index('longitude'))
dropdown_y = ipywidgets.Dropdown(options=list(ufosDS.columns),index=list(ufosDS.columns).index('latitude'))
dropdown_color = ipywidgets.Dropdown(options=plt.colormaps())

x_select = ipywidgets.Box([ipywidgets.Label(value = 'x:'),dropdown_x])
y_select = ipywidgets.Box([ipywidgets.Label(value = 'y:'),dropdown_y])
color_select = ipywidgets.Box([ipywidgets.Label(value = 'c:'),dropdown_color])

controls = ipywidgets.VBox([x_select, y_select,color_select])
controls.layout.top = '200px'
hbox=ipywidgets.HBox([out, controls])
display(hbox)

# change the x and y fields
def updateColumns(change):
    if change is not None:
        # grab names of columns from other widget
        x = list(ufosDS.columns)[dropdown_x.index]
        y = list(ufosDS.columns)[dropdown_y.index]
        cmap = plt.colormaps()[dropdown_color.index]
        for a in fig.axes:
            fig.delaxes(a)       
        ax = fig.add_axes([0.0, 0.15, 1.0, 0.8]) 
        # use the log of the duration in seconds as the color scaling
        image = ax.scatter(ufosDS[x],ufosDS[y],c=np.log10(ufosDS['duration_seconds']),cmap = cmap)
        # modify some charts that cannot be fully displayed
        if x == 'shape' or x=='state':
            plt.xticks(rotation=30)            
    with out:
        clear_output(wait=True)
        display(fig)

# change color map
def updateColors(change):
    if change is not None:
        # grab name of color maps from other widget
        cmap=plt.colormaps()[change['owner'].index]
        for a in fig.axes:
            fig.delaxes(a)
        ax = fig.add_axes([0.0, 0.15, 1.0, 0.8])
        # use the log of the duration in seconds as the color scaling
        image = ax.scatter(ufosDS[dropdown_x.value],ufosDS[dropdown_y.value],c=np.log10(ufosDS['duration_seconds']),cmap = cmap)
        # modify some charts that cannot be fully displayed
        if dropdown_x.value == 'shape' or dropdown_x.value=='state':
            plt.xticks(rotation=30)            
    with out:
        clear_output(wait=True)
        display(fig)     

dropdown_x.observe(updateColumns)
dropdown_y.observe(updateColumns)
dropdown_color.observe(updateColors)
updateColumns(None)
updateColors(None)

<font size = 3>The visualization above allows user to choose the x-axis and y-axis of the scatter plot, but not all of the columns in the UFO dataset can be chosen. I removed three categorical variables —— city, comment, duration —— from both of the dropdown_x and dropdown_y. The specific reasons are much same as above, here is some explains from Linyao: <br>
(1) City: There are 19900 unique cities in the dataset. The quantity is so large that it will cause the axis label overlaps so that they cannot be shown well on the chart.<br>
(2) Comment: Almost every record has a unique comment, which not only makes it difficult to arrange charts, but also causes the scatter plot to be meaningless.<br>
(3) Duration: This variable seems to be a numerical variable. However, in this dataset, it is described in natural language with different forms and different scales such as '3 minutes or less', 'approx. 35 mins', and '1-2 hrs' , which makes it categorical. There are 8349 unique durations in the dataset. Therefore, it is also a large number that makes it hard to draw a clear chart, but things will be different if we can convert this variable to numerical variable under uniform measures.<font>